In [39]:
from google.colab import files
import zipfile
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import SVR

In [10]:
subject_info=files.upload()
proccessed_data=files.upload()

Saving subject-info.xlsx to subject-info (1).xlsx


Saving proccessed_data.zip to proccessed_data.zip


In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [13]:


def mean_jump(file_path):
    # Load data from CSV file
    df = pd.read_csv(file_path)

    # Extract time and data from DataFrame
    time = df["Time [s]"]
    data = df["V_tidal [L]"]

    # Find local maxima and minima
    local_maxima = (np.diff(np.sign(np.diff(data))) < 0).nonzero()[0] + 1
    local_minima = (np.diff(np.sign(np.diff(data))) > 0).nonzero()[0] + 1

    # Ensure the number of maxima and minima are the same
    min_length = min(len(local_maxima), len(local_minima))
    local_maxima = local_maxima[:min_length]
    local_minima = local_minima[:min_length]

    # Calculate differences between consecutive maxima and minima
    differences = data.iloc[local_maxima].values - data.iloc[local_minima].values

    # Calculate average of differences
    average_difference = np.mean(differences)

    return average_difference

In [15]:
zip_filename = 'proccessed_data.zip'
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('processed_data')


parent_directory_path = 'processed_data'

nested_directory_path = os.path.join(parent_directory_path, 'proccessed_data')

# Get a list of CSV files sorted by their names
csv_files = [file_name for file_name in os.listdir(nested_directory_path)]
csv_files.sort()

# Iterate through each CSV file, calculate the average, and store the results
averages = []
for file_name in csv_files:
    file_path = os.path.join(nested_directory_path, file_name)
    average_v_tidal = mean_jump(file_path)
    averages.append({'File': file_name, 'Average_V_tidal': average_v_tidal})

average_values = [entry['Average_V_tidal'] for entry in averages]
# Create a DataFrame from the averages list
data=pd.read_excel("subject-info (1).xlsx")

tidal_volume_series = pd.Series(average_values, name='tidal_volume')

# Assign the Series to the 'tidal_volume' column
data['tidal_volume'] = tidal_volume_series

data.to_excel("final_data_set.xlsx", index=False)

files.download("final_data_set.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:


data['Sex (M/F)']= data['Sex (M/F)'].replace('M',-1)
data['Sex (M/F)']= data['Sex (M/F)'].replace('F',1)
data['Asthma (Y/N)'] = data['Asthma (Y/N)'].replace('Y',1)
data['Asthma (Y/N)'] = data['Asthma (Y/N)'].replace('N',0)
data['History of Smoking (Y/N)'] = data['History of Smoking (Y/N)'].replace('N',0)
data['History of Smoking (Y/N)'] = data['History of Smoking (Y/N)'].replace('Y',1)
data['History of Vaping (Y/N)'] = data['History of Vaping (Y/N)'].replace('N',0)
data['History of Vaping (Y/N)'] = data['History of Vaping (Y/N)'].replace('Y',1)
data.replace('M ', -1, inplace=True)

# Feature selection/engineering
X = data[['Age [years]', 'Weight [kg]', 'Height [cm]', 'Chest Depth [mm]', 'Chest Width [mm]',
          'Sex (M/F)', 'Asthma (Y/N)', 'History of Smoking (Y/N)', 'History of Vaping (Y/N)']]
y = data['tidal_volume']



In [19]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=1)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Model evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)



Mean Absolute Error: 0.2675275531904294


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Define a list of values for n_estimators to experiment with
n_estimators_values = [50, 100, 150, 200, 250,350,450,550,650,750,850,950,1000]

# Iterate over each value of n_estimators
for n_estimators in n_estimators_values:
    # Instantiate the Random Forest Regressor model with the current value of n_estimators
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the MAE for the current value of n_estimators
    print(f"n_estimators = {n_estimators}, Mean Absolute Error: {mae}")

n_estimators = 50, Mean Absolute Error: 0.21233410828171217
n_estimators = 100, Mean Absolute Error: 0.20414702504280893
n_estimators = 150, Mean Absolute Error: 0.2073048123279492
n_estimators = 200, Mean Absolute Error: 0.20476390966409957
n_estimators = 250, Mean Absolute Error: 0.20093678466861872
n_estimators = 350, Mean Absolute Error: 0.20038107251243692
n_estimators = 450, Mean Absolute Error: 0.19903799606762307
n_estimators = 550, Mean Absolute Error: 0.20164575124640163
n_estimators = 650, Mean Absolute Error: 0.20457524386356885
n_estimators = 750, Mean Absolute Error: 0.20535197750416914
n_estimators = 850, Mean Absolute Error: 0.20639606421081502
n_estimators = 950, Mean Absolute Error: 0.2048977076787292
n_estimators = 1000, Mean Absolute Error: 0.2050708515568506


In [21]:

# Define a function to perform input and output data augmentation
def augment_data(X_input, y_output,perturbation_factor):
    augmented_X_input = []
    augmented_y_output = []
    for input_data, output_data in zip(X_input, y_output):
        # Augment input data
        perturbed_input = input_data + np.random.normal(scale=perturbation_factor, size=input_data.shape)
        augmented_X_input.append(perturbed_input)
        # Output data remains unchanged during augmentation
        augmented_y_output.append(output_data)
    return np.array(augmented_X_input), np.array(augmented_y_output)

In [34]:
# Assuming X and y are currently strings or in a format that is not compatible with the function
X_array = np.array(X)  # Convert X to numpy array
y_array = np.array(y)  # Convert y to numpy array

# Now call the augment_data function
X_augmented, y_augmented = augment_data(X_array, y_array, perturbation_factor=0.1)


X_train=X
y_train=y
X_test = pd.DataFrame(X_augmented)
y_test = pd.DataFrame(y_augmented)

We did augmentation here in order to increase our dataset thereby increasing our accuracy

In [31]:

n_estimators_values = [50, 100, 150, 200, 250,350,450,550,650,750,850,950,1000]

# Iterate over each value of n_estimators
for n_estimators in n_estimators_values:
    # Instantiate the Random Forest Regressor model with the current value of n_estimators
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the MAE for the current value of n_estimators
    print(f"n_estimators = {n_estimators}, Mean Absolute Error: {mae}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


n_estimators = 50, Mean Absolute Error: 0.10028600920925321


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


n_estimators = 100, Mean Absolute Error: 0.10278071610033494


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


n_estimators = 850, Mean Absolute Error: 0.10337309560025168


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


n_estimators = 950, Mean Absolute Error: 0.10325500993070738
n_estimators = 1000, Mean Absolute Error: 0.1037772569759186


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [38]:
X_train.columns = [str(col).replace("[", "").replace("]", "").replace("<", "") for col in X.columns]
X_test.columns = [str(col).replace("[", "").replace("]", "").replace("<", "") for col in X.columns]

# Instantiate the XGBoost regressor model
model = xgb.XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.133875599435572


In [40]:
model = SVR(kernel='rbf')  # You can choose different kernels like 'linear', 'poly', 'rbf', etc.

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.2670534077733942
